

1. ***Randon Forest***


In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

**Mounting G drive**


> You need to authenticate to GD in order to get the AWID dataset with the scripts.



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Get the resources**

> Get all necessary resources from Google Drive, this works after the authentification.these resources are compressed to save some time.


In [0]:
!cp -rn "/content/drive/My Drive/AWID/AWID-CLS-R-Trn_with_headers.zip" .
!cp -rn "/content/drive/My Drive/AWID/AWID-CLS-R-Tst_with_headers.zip" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/utils.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/main.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/imports.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/class_report.py" .

!unzip AWID-CLS-R-Trn_with_headers.zip 
!unzip AWID-CLS-R-Tst_with_headers.zip 
!rm *.zip
!rm -rf sample_data/

Archive:  AWID-CLS-R-Trn_with_headers.zip
  inflating: AWID-CLS-R-Trn/AWID-CLS-R-Trn_with_headers.csv  
Archive:  AWID-CLS-R-Tst_with_headers.zip
  inflating: AWID-CLS-R-Tst/AWID-CLS-R-Tst_with_headers.csv  


***Imports***


> Import all necessary libraries.



In [0]:
from imports import *

***Read the data and replace the '?' string in the Pandas DataFrame with a NumPy NaN value***

In [0]:
train_path = '/content/AWID-CLS-R-Trn/AWID-CLS-R-Trn_with_headers.csv'
test_path = '/content/AWID-CLS-R-Tst/AWID-CLS-R-Tst_with_headers.csv'
train_df_raw = pd.read_csv(train_path, low_memory=False)
test_df_raw = pd.read_csv(test_path, low_memory=False)
train_df_raw.replace('?', np.nan, inplace=True)
test_df_raw.replace('?', np.nan, inplace=True)


**Drop these colomns**


> Each one of the dropped columns has a Nan rate of 1.
> Note that we need to drop them from both of training and validation sets.


In [0]:
del test_df_raw['frame.dlt']
del test_df_raw['wlan.qos.buf_state_indicated']
del test_df_raw['radiotap.mactime']
del test_df_raw['wlan.ba.bm']
del test_df_raw['wlan.ba.control.ackpolicy']
del test_df_raw['wlan.ba.control.cbitmap']
del test_df_raw['wlan.ba.control.multitid']
del test_df_raw['wlan.bar.compressed.tidinfo']
del test_df_raw['wlan.bar.type']
del test_df_raw['wlan_mgt.fixed.fragment']
del test_df_raw['wlan_mgt.fixed.sequence']
del test_df_raw['wlan_mgt.tcprep.link_mrg']
del test_df_raw['wlan_mgt.tcprep.trsmt_pow']


del train_df_raw['frame.dlt']
del train_df_raw['wlan.qos.buf_state_indicated']
del train_df_raw['radiotap.mactime']
del train_df_raw['wlan.ba.bm']
del train_df_raw['wlan.ba.control.ackpolicy']
del train_df_raw['wlan.ba.control.cbitmap']
del train_df_raw['wlan.ba.control.multitid']
del train_df_raw['wlan.bar.compressed.tidinfo']
del train_df_raw['wlan.bar.type']
del train_df_raw['wlan_mgt.fixed.fragment']
del train_df_raw['wlan_mgt.fixed.sequence']
del train_df_raw['wlan_mgt.tcprep.link_mrg']
del train_df_raw['wlan_mgt.tcprep.trsmt_pow']


***Data pre-processing***


> We need to treat the data before passing it to our model.



***Change any columns of strings in a panda's dataframe to a column of categorical values.***
 

> This applies the changes inplace.




In [0]:
str_to_cat(train_df_raw)

***Changes any columns of strings in df into categorical variables using trn as a template for the category codes***
 

> This applies the changes inplace too.




In [0]:
apply_cats(test_df_raw,train_df_raw)

***We're still not quite done !***
 
> For instance we have lots of missing values, which we can't pass directly to the model.




***We call  the function proc_df() to process a pandas dataframe !***
 

*   *proc_df*  takes a data frame df and splits off the response variable in our case the *class* colomn, and changes the df into an entirely numeric dataframe. For each column of df which is not in skip_flds nor in ignore_flds, na values are replaced by the median value of the column.
*    Three other funtions are called inside this function:
    *    ***numericalize()*** : Changes the column col from a categorical type to it's integer codes.
    *    ***fix_missing()*** : Fill missing data in a column of df with the median, and add a {name}_na column 
    which specifies if the data was missing.
    *    ***scale_vars()*** : Scale tha data.



In [0]:
x_train, y_train , nas , mapper = proc_df(train_df_raw, 'class', do_scale=True)
x_valid, y_valid , nas , _ = proc_df(test_df_raw,'class', na_dict=nas, do_scale=True, mapper=mapper)

***Classification with the Random forest ***

Instantiate and train the model

In [0]:
rf = RandomForestClassifier(n_jobs=-1, n_estimators=100)
%time rf.fit(x_train, y_train) 

CPU times: user 5min 59s, sys: 585 ms, total: 6min
Wall time: 1min 33s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

Evaluate the model:

In [0]:
# making predictions on the testing set 
predict = rf.predict(x_valid)
CM = confusion_matrix(y_valid, predict)
CR = classification_report (y_valid,predict)
report_with_auc = class_report(y_true=y_valid, y_pred=predict,y_score=rf.predict_proba(x_valid))
print("************************the original set *************************")
print("=== Confusion Matrix ===\n",CM)
print("\n=== Report with classification report  ===\n\n",CR)
print("\n=== Report with auc  ===\n\n",report_with_auc)

************************the original set *************************
=== Confusion Matrix ===
 [[  5291      0      0   2806]
 [     0   1471      0  18608]
 [     0      0  13811   2871]
 [     1      2      0 530781]]

=== Report with classification report  ===

               precision    recall  f1-score   support

           0       1.00      0.65      0.79      8097
           1       1.00      0.07      0.14     20079
           2       1.00      0.83      0.91     16682
           3       0.96      1.00      0.98    530784

    accuracy                           0.96    575642
   macro avg       0.99      0.64      0.70    575642
weighted avg       0.96      0.96      0.94    575642


=== Report with auc  ===

              precision    recall  f1-score   support      pred       AUC
0             0.999811  0.653452  0.790350    8097.0    5292.0  0.833700
1             0.998642  0.073261  0.136507   20079.0    1473.0  0.804034
2             1.000000  0.827898  0.905847   16682.0  

***features selection***
> We are going to test our model in different sets.

> Want to know more about these sets? The details are in the report.





In [0]:
features2 = ['radiotap.datarate', 'wlan.fc.type_subtype', 'wlan.seq','wlan_mgt.fixed.capabilities.preamble','wlan_mgt.fixed.timestamp','wlan_mgt.fixed.beacon','wlan_mgt.tim.dtim_period','data.len','class']
features3 = ['frame.time_epoch' , 'frame.time_relative' , 'radiotap.datarate' ,'wlan.fc.type_subtype' ,'wlan.fc.ds' ,'wlan.fc.retry' ,'wlan.fc.protected' ,'wlan.ta' ,'wlan.seq' ,'wlan_mgt.fixed.capabilities.ess' ,'wlan_mgt.fixed.capabilities.preamble' ,'wlan_mgt.fixed.capabilities.short_slot_time' ,'wlan_mgt.fixed.timestamp' ,'wlan_mgt.fixed.beacon' ,'wlan_mgt.fixed.auth.alg' ,'wlan_mgt.fixed.auth_seq' ,'wlan_mgt.tim.dtim_period' ,'wlan_mgt.rsn.akms.type' ,'wlan.wep.key' ,'data.len' ,'class']
features4 = ['frame.time_epoch','frame.time_delta','frame.time_delta_displayed','frame.time_relative','frame.len','frame.cap_len','wlan.fc.subtype','wlan.fc.ds','wlan.duration','wlan.ra','wlan.da','wlan.ta','wlan.sa','wlan.seq','wlan.wep.iv','wlan.wep.icv','data.len','class']

# set 2
train2 = train_df_raw[features2]
test2 = test_df_raw[features2]  
str_to_cat(train2)
apply_cats(test2,train2)
x_valid2 , y_valid2 , nas2 , mapper2 = proc_df(test2, 'class' , do_scale = True)
x_train2 , y_train2 , nas2 , _ = proc_df(train2, 'class',na_dict=nas2 , do_scale = True , mapper = mapper2)

# set 3
train3 = train_df_raw[features3]
test3 = test_df_raw[features3] 
str_to_cat(train3)
apply_cats(test3,train3)
x3,y3,nas3 = proc_df(train3, 'class')
x_valid3 , y_valid3 , nas3 , mapper3 = proc_df(test3,'class' , do_scale = True)
x_train3 , y_train3 , nas3 , _ = proc_df(train3, 'class',na_dict=nas3 , do_scale = True , mapper = mapper3)

# set 4
train4 = train_df_raw[features4]
test4 = test_df_raw[features4] 
str_to_cat(train4)
apply_cats(test4,train4)
x4,y4,nas4 = proc_df(train4, 'class')
x_valid4 , y_valid4 , nas4 , mapper4 = proc_df(test4,'class' , do_scale = True)
x_train4 , y_train4 , nas4 , _ = proc_df(train4, 'class',na_dict=nas4 , do_scale = True , mapper = mapper4)



/content/utils.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()
/content/utils.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n] = c.astype('category').cat.as_ordered()
/content/utils.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [0]:
# set2
rf2 = RandomForestClassifier(n_jobs=-1, n_estimators=100)
%time rf2.fit(x_train2, y_train2) 
  
# making predictions on the testing set 
predict2 = rf2.predict(x_valid2)
CM2 = confusion_matrix(y_valid2, predict2)
CR2 = classification_report (y_valid2,predict2)
report_with_auc2 = class_report(y_true=y_valid2, y_pred=predict2,y_score=rf2.predict_proba(x_valid2))
print("************************set2 *************************")
print("=== Confusion Matrix ===\n",CM2)
print("\n=== Report with auc  ===\n\n",report_with_auc2)

CPU times: user 3min 12s, sys: 416 ms, total: 3min 13s
Wall time: 49.5 s
************************set2 *************************
=== Confusion Matrix ===
 [[  1076      0      0   7021]
 [     0   1467      0  18612]
 [     0     26  13878   2778]
 [  2877   3546  15587 508774]]

=== Report with auc  ===

              precision    recall  f1-score   support      pred       AUC
0             0.272198  0.132889  0.178589    8097.0    3953.0  0.776397
1             0.291129  0.073061  0.116809   20079.0    5039.0  0.491581
2             0.470999  0.831915  0.601469   16682.0   29465.0  0.912829
3             0.947111  0.958533  0.952788  530784.0  537185.0  0.681431
avg / total   0.900939  0.912364  0.902557  575642.0  575642.0  0.966974


In [0]:
# set3
rf3 = RandomForestClassifier(n_jobs=-1, n_estimators=100)
%time rf3.fit(x_train3, y_train3) 
  
# making predictions on the testing set 
predict3 = rf3.predict(x_valid3)
CM3 = confusion_matrix(y_valid3, predict3)
CR3 = classification_report (y_valid3,predict3)
report_with_auc3 = class_report(y_true=y_valid3, y_pred=predict3,y_score=rf3.predict_proba(x_valid3))
print("************************set3 *************************")
print("=== Confusion Matrix ===\n",CM3)
print("\n=== Report with auc  ===\n\n",report_with_auc3)

CPU times: user 3min 14s, sys: 1.27 s, total: 3min 16s
Wall time: 50.5 s
************************set3 *************************
=== Confusion Matrix ===
 [[   708      0      0   7389]
 [     0    734      0  19345]
 [     0      0  13790   2892]
 [     0      0      0 530784]]

=== Report with auc  ===

              precision    recall  f1-score   support      pred       AUC
0             1.000000  0.087440  0.160818    8097.0     708.0  0.861014
1             1.000000  0.036556  0.070533   20079.0     734.0  0.842189
2             1.000000  0.826639  0.905093   16682.0   13790.0  0.999890
3             0.947135  1.000000  0.972850  530784.0  560410.0  0.898671
avg / total   0.951255  0.948534  0.927991  575642.0  575642.0  0.990416


In [0]:
# set4
rf4 = RandomForestClassifier(n_jobs=-1, n_estimators=100)
%time rf4.fit(x_train4, y_train4) 
  
# making predictions on the testing set 
predict4 = rf4.predict(x_valid4)
CM4 = confusion_matrix(y_valid4, predict4)
CR4 = classification_report (y_valid4,predict4)
report_with_auc4 = class_report(y_true=y_valid4, y_pred=predict4,y_score=rf4.predict_proba(x_valid4))
print("************************set4 *************************")
print("=== Confusion Matrix ===\n",CM4)
print("\n=== Report with auc  ===\n\n",report_with_auc4)

CPU times: user 7min 8s, sys: 1.45 s, total: 7min 9s
Wall time: 1min 50s
************************set4 *************************
=== Confusion Matrix ===
 [[  4168      0      0   3929]
 [     0   1292      0  18787]
 [     0      0  13811   2871]
 [     2      0      0 530782]]

=== Report with auc  ===

              precision    recall  f1-score   support      pred       AUC
0             0.999520  0.514759  0.679547    8097.0    4170.0  0.861335
1             1.000000  0.064346  0.120912   20079.0    1292.0  0.819621
2             1.000000  0.827898  0.905847   16682.0   13811.0  0.991336
3             0.954011  0.999996  0.976462  530784.0  556369.0  0.955624
avg / total   0.957588  0.955547  0.940397  575642.0  575642.0  0.989381


***Save the models***

> With the ***dill*** framework,which is a utility for serialization of python objects.



In [0]:
os.makedirs("RF_Models", exist_ok=True)

dill.dump(rf, open("RF_Models/rf.obj", "wb")) 
dill.dump(rf2, open("RF_Models/rf2.obj", "wb"))
dill.dump(rf3, open("RF_Models/rf3.obj", "wb"))
dill.dump(rf4, open("RF_Models/rf4.obj", "wb"))

In [20]:
!zip -r RF_Models.zip RF_Models
!cp -f RF_Models.zip "/content/drive/My Drive/Mahamdi's PFE/Models"

updating: RF_Models/ (stored 0%)
  adding: RF_Models/.ipynb_checkpoints/ (stored 0%)
  adding: RF_Models/rf.obj (deflated 80%)
  adding: RF_Models/rf4.obj (deflated 79%)
  adding: RF_Models/rf2.obj (deflated 81%)
  adding: RF_Models/rf3.obj (deflated 79%)
